# VisB


Initially VisB was developed for ProB2-UI, the new user interface of ProB based on JavaFX and the ProB2-Java-API.
It is included in ProB2-UI version 1.1.0 and later.


ProB Tcl/Tk and probcli version 1.11.0 and later also support VisB visualisations:

* with <tt>probcli File.mch ... -visb VISBJSONFILE HTMLFILE</tt> you can export the animator's history to a self-contained HTML file including a visualisation of the states in the history
* in ProB Tcl/Tk you can visualise the current state or the history using a VisB visualisation file (it is shown in an external browser) by right clicking in the "State Properties" or the "History" pane.
Hovers work in both cases, but unlike in ProB2-UI, you cannot click to perform events.

An article about VisB has been [https://rdcu.be/b4rqh published in the ABZ'2020 proceedings].


As of version 1.12 you can also provide many of the VisB annotations via B DEFINITIONS.
This can be done in addition to a VisB JSON file or completely replacing the JSON file.

By adding this definition to your B machine you specify that you wish to use an empty
SVG file as base:
<pre>
  VISB_JSON_FILE == "";
</pre>

### VISB_SVG_BOX

With a VISB_SVG_BOX definition you can set the size of the generated empty SVG file:
<pre>
  VISB_SVG_BOX == rec(height:formula, width:1800);
</pre>

Note: you can also provide an optional <tt>viewBox</tt> attribute as a B string.

### VISB_SVG_OBJECTS

You can populate the empty SVG file with your own new objects using DEFINITIONS that start with <pre>VISB_SVG_OBJECTS</pre>.
Here is an example where we create one red circle:

In [1]:
::load
MACHINE test1
DEFINITIONS
  VISB_JSON_FILE == "";
  VISB_SVG_BOX == rec(height:100, width:400);
  VISB_SVG_OBJECTS == rec(`id`:"button", svg_class:"circle",
     cx:200,cy:50, r:30, fill: "red",
     stroke:"black", `stroke-width`:2);
END

Loaded machine: test1

In [2]:
:init

Executed operation: INITIALISATION()

In [3]:
:show

<VisB visualization>

A VISB_SVG_OBJECT definition should either define a record or a set of records. The fields of the record specify the kind of object created and its initial attributes:
* usually all created object should have an id (so that you can later modify the attributes)
* you have to provide an <tt>svg_class</tt> attribute: circle, ellipse, line, path, polygon, polyline, rect, text
* you then can (and probably) should provide necessary attributes and initial values for your object

Note: <tt>id</tt> is a keyword in B and hence you have to use backquotes around it. Similarly, B does not allow identifiers with hyphens, hence you need surround such attributes (like <tt>stroke-width</tt>) with backquotes as well.

In [4]:
::load
MACHINE test_svg_objects
DEFINITIONS
  N == 5; WID==400.0;
  VISB_JSON_FILE == "";
  VISB_SVG_BOX == rec(height:100, width:WID+30.0);
  VISB_SVG_OBJECTS == UNION(i).(i:1..N|
    {rec(`id`:("circ",i), svg_class:"circle",
     cx:real(i)*WID/real(N),cy:50, r:(30-N), fill: "red",
     stroke:"black", `stroke-width`:2)});
END

Loaded machine: test_svg_objects

In [5]:
:init

Executed operation: INITIALISATION()

In [6]:
:show

<VisB visualization>

In [7]:
:table VISB_SVG_OBJECTS

|cx|cy|fill|id|r|stroke|stroke-width|svg_class|
|---|---|---|---|---|---|---|---|
|80.0|50|"red"|("circ"↦1)|25|"black"|2|"circle"|
|160.0|50|"red"|("circ"↦2)|25|"black"|2|"circle"|
|240.0|50|"red"|("circ"↦3)|25|"black"|2|"circle"|
|320.0|50|"red"|("circ"↦4)|25|"black"|2|"circle"|
|400.0|50|"red"|("circ"↦5)|25|"black"|2|"circle"|


### VISB_SVG_UPDATES
To dynamically update the SVG objects based on a B model's state you can specify <tt>VISB_SVG_UPDATES</tt> DEFINITIONS. They have the same form as <tt>VISB_SVG_OBJECTS</tt> DEFINITIONS, except you should not specify the <tt>svg_class</tt> attribute. In addition, the attribute values can depend on B variables.

Here is a small example:

In [8]:
::load
MACHINE test_svg_updates
VARIABLES x
INVARIANT x:1..N
INITIALISATION x:=1
OPERATIONS inc = BEGIN x:= (x mod N)+1 END
DEFINITIONS
  N == 5; WID==400.0;
  VISB_JSON_FILE == "";
  VISB_SVG_BOX == rec(height:100, width:WID+30.0);
  VISB_SVG_OBJECTS == UNION(i).(i:1..N|
    {rec(`id`:("circ",i), svg_class:"circle",
     cx:real(i)*WID/real(N),cy:50, r:(30-N),
     stroke:"black")});
  VISB_SVG_UPDATES == UNION(i).(i:1..N|
    {rec(`id`:("circ",i),
     fill: IF i=x THEN "red" ELSE "gray" END,
     `stroke-width`:IF i=x THEN 2.0 ELSE 0.5 END)});
END

Loaded machine: test_svg_updates

In [9]:
:init

Executed operation: INITIALISATION()

In [10]:
:show

<VisB visualization>

In [11]:
:exec inc

Executed operation: inc()

In [12]:
:show

<VisB visualization>

In [13]:
:table VISB_SVG_UPDATES

|fill|id|stroke-width|
|---|---|---|
|"gray"|("circ"↦1)|0.5|
|"gray"|("circ"↦3)|0.5|
|"gray"|("circ"↦4)|0.5|
|"gray"|("circ"↦5)|0.5|
|"red"|("circ"↦2)|2.0|


### VISB_SVG_CONTENTS

With <tt>VISB_SVG_CONTENTS</tt> DEFINITIONS you can add additional static content to your SVG file. This is useful for CSS styles or arrow markers, for example.
Here is an example:

In [14]:
::load
MACHINE test_svg_contents
VARIABLES x
INVARIANT x:1..N
INITIALISATION x:=1
OPERATIONS inc = BEGIN x:= (x mod N)+1 END
DEFINITIONS
  N == 5; WID==400.0;
  VISB_JSON_FILE == "";
  VISB_SVG_BOX == rec(height:100, width:WID+30.0);
  VISB_SVG_OBJECTS == UNION(i).(i:1..N|
    {rec(`id`:("circ",i), svg_class:"circle",
     cx:real(i)*WID/real(N),cy:50, r:(30-N))});
  VISB_SVG_UPDATES == UNION(i).(i:1..N|
    {rec(`id`:("circ",i),
     class: IF i=x THEN "selected" ELSE "normal" END)});
  VISB_SVG_CONTENTS == '''
  <style>
        .selected {
            fill: red;
            stroke : black;
            stroke-width: 2.0
        }
        .normal {
            fill : gray;
            stroke : none
        }
 </style>
''';
END

Loaded machine: test_svg_contents

In [15]:
:init

Executed operation: INITIALISATION()

In [16]:
:show

<VisB visualization>

In [17]:
:table VISB_SVG_UPDATES

|class|id|
|---|---|
|"normal"|("circ"↦2)|
|"normal"|("circ"↦3)|
|"normal"|("circ"↦4)|
|"normal"|("circ"↦5)|
|"selected"|("circ"↦1)|


In [18]:
:table VISB_SVG_OBJECTS

|cx|cy|id|r|svg_class|
|---|---|---|---|---|
|80.0|50|("circ"↦1)|25|"circle"|
|160.0|50|("circ"↦2)|25|"circle"|
|240.0|50|("circ"↦3)|25|"circle"|
|320.0|50|("circ"↦4)|25|"circle"|
|400.0|50|("circ"↦5)|25|"circle"|


### VISB_SVG_FILE

It is possible to combine the above VisB DEFINITIONS with either an additional SVG graphics file or with an additional JSON file (which can itself include an SVG graphics file).

Note that when creating an HTML export the file includes an SVG description inside.
For example, the above rendering contains inside this SVG code:
```
<svg  xmlns="http://www.w3.org/2000/svg"
      width="430.0" height="100" viewBox="0 0 430.0 100" >
  <style>
        .selected {
            stroke : black;
            stroke-width: 2.0;
            fill: red
        }
        .normal {
            stroke : none;
            fill : white
        }
   </style>
 <circle id="circ1" class="normal" cx="80.0" cy="50" r="29"></circle>
 <circle id="circ2" class="normal" cx="160.0" cy="50" r="28"></circle>
 <circle id="circ3" class="selected" cx="240.0" cy="50" r="27"></circle>
 <circle id="circ4" class="normal" cx="320.0" cy="50" r="26"></circle>
 <circle id="circ5" class="normal" cx="400.0" cy="50" r="25"></circle>
</svg>
```

Below we have put this SVG into the file five_circles.svg and have modified it slightly, making the circles smaller and changing the color scheme. Note: you can leave the above VISB_SVG_OBJECTS definitions in the B machine: the SVG file will take precedence; only in case the SVG does not contain an object of the specified id will it be added.

In [19]:
::load
MACHINE test_svg_file
VARIABLES x
INVARIANT x:1..N
INITIALISATION x:=1
OPERATIONS inc = BEGIN x:= (x mod N)+1 END
DEFINITIONS
  N == 5; WID==400.0;
  VISB_SVG_FILE == "five_circles.svg";
  VISB_SVG_BOX == rec(height:100, width:WID+30.0);
  VISB_SVG_OBJECTS == UNION(i).(i:1..N|
    {rec(`id`:("circ",i), svg_class:"circle",
     cx:real(i)*WID/real(N),cy:50, r:(30-N))});
  VISB_SVG_UPDATES == UNION(i).(i:1..N|
    {rec(`id`:("circ",i),
     class: IF i=x THEN "selected" ELSE "normal" END)});
END

Loaded machine: test_svg_file

In [20]:
:init

Executed operation: INITIALISATION()

In [21]:
:show

<VisB visualization>

In [22]:
x

$1$

In [23]:
:exec inc

Executed operation: inc()

In [24]:
:show

<VisB visualization>